# 3 задача
## Найти собственные значения задачи Штурма-Лиувилля

### Постановка задачи

\begin{equation}
\left.
  \begin{array}{ccc}
    \frac{d^2y}{dx^2}+\lambda q_0(x)y = 0\\
   y(0) = y(1) = 0
  \end{array}
\right\}
\end{equation}

\begin{equation}
q_0(x) =  
\left\{
  \begin{array}{ccc}
    2, 0 \leq x \leq 1/2 \\
    1, 1/2 \leq x \leq 1 
  \end{array}
\right.
\end{equation}

\begin{equation}
0 < x < D = 1
\end{equation}



In [162]:
x0, xN = 0, 1      # концы отрезка
L = 1000             # число отрезков
h = (xN - x0) / L  # шаг сетки

### Заменим производные в узлах сетки конечными разностями

\begin{equation} 
\left\{
  \begin{array}{ccc}
      \frac{y_{l-1} - 2y_l + y_{l+1}}{h^2} + \lambda q_{0_l} y_l = 0\\
      y_0 = y_L = 0
  \end{array}
\right.
\end{equation}

#### Теперь необходимо решить полученную однородную систему линейных уравнений

Перепишем систему в более удобном виде:

\begin{equation} 
\left\{
  \begin{array}{ccc}
      y_{l-1} + (\lambda h^2 q_{0_l} - 2)y_l + y_{l+1} = 0\\
      y_0 = y_L = 0
  \end{array}
\right.
\end{equation}

Размер матрицы системы: L+1 на L+1

Для решения такой системы (она имеет тривиальное решение при невырожденности, нам это не нужно!) необходимо равенство нуля детерминанта матрицы системы A:
\begin{equation} 
A(\lambda)\cdot \overrightarrow{y} = \overrightarrow{0}
\end{equation}

#### Заметим, что матрица системы имеет удобный для численного решения вид - трёхдиагональный.
Для таких матриц используется метод прогонки (алгоритм Томаса). Мы же воспользуемся встроенной функцией из numpy.linalg для разреженной матрицы из scipy.sparse:

In [163]:
import numpy as np
from scipy.sparse import dia_matrix
from numpy import linalg as LA

def q0(x):
    return 2 if x <= 1/2 else 1

def calcDet(lam):
    upper_arr = [0, 0] + [1 for i in range(1, L)]
    middle_arr = [1] + [-2+lam* q0(i*h) *h**2 for i in range(1, L)] + [1]
    lower_arr = [1 for i in range(1, L)] + [0, 0]
    return LA.det(dia_matrix((np.array([lower_arr, middle_arr, upper_arr]), [-1, 0, 1]), shape=(L + 1, L + 1)).toarray())


### Локализуем корни
#### Для этого найдем области смены знака уравнения 
\begin{equation} 
det(A(\lambda)) = 0
\end{equation}

In [170]:
n = 100
z1, z2 = 0, 200
value = [calcDet(z1 + i * (z2 - z1) / n) for i in range(n + 1)]
roots = []
for i in range(n):
#     print(value[i] * value[i + 1])
    if value[i] * value[i + 1] < 0:
        x1 = z1 + i * (z2 - z1) / n
        x2 = z1 + (i + 1) * (z2 - z1) / n
        roots.append((x1, x2))
print('Локализуем корни:')
for i in range(len(roots)): print(roots[i])

Локализуем корни:
(6.0, 8.0)
(28.0, 30.0)
(58.0, 60.0)
(110.0, 112.0)
(166.0, 168.0)


### Уточняем значение корня методом половинного деления
#### Для каждого корня повторяем процедуру до достижения целевой точности 
\begin{equation} 
\delta = 10^{-4}
\end{equation}

In [172]:
delta = 1e-4
Y = []
for i in range(len(roots)):
    a = roots[i][0]
    b = roots[i][1]
    while (b - a > delta):
        c = (a + b) / 2
        if calcDet(c) * calcDet(a) < 0:
            b = c
        else:
            a = c
    Y.append((a + b) / 2)
print('\nУточняем значение корней:')
for i in range(len(Y)): print(Y[i])


Уточняем значение корней:
6.403717041015625
28.431304931640625
58.730743408203125
110.76235961914062
167.68771362304688
